### SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("default")
    .getOrCreate()
)

#https://www.bcb.gov.br/api/servico/sitebcb/historicotaxajurosdiario/TodosCampos?filtro=(codigoSegmento%20eq%20%272%27)%20and%20(codigoModalidade%20eq%20%27211101%27)%20and%20(InicioPeriodo%20eq%20%272025-03-24%27)

### Data única

In [ ]:

from utils.params import PRODUTOS, PRODUTOS_TST
from utils.helper import *
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, TimestampType

df_list = extrair_dados_bacen_data_unica(PRODUTOS, data_alvo="2025-03-24")

schema = StructType([
        StructField("NM_BANK", StringType(), True),
        StructField("NM_SEGM", StringType(), True),
        StructField("DS_PROD", StringType(), True),
        StructField("CD_SEGM", StringType(), True),
        StructField("CD_MODL", StringType(), True),
        StructField("NR_POSI", IntegerType(), True),
        StructField("DS_MODL", StringType(), True),
        StructField("DT_ALVO", StringType(), True),
        StructField("IN_INIC_PERI_EXAT", IntegerType(), True),
        StructField("DT_APROX", StringType(), True),
        StructField("QT_DIA_APROX", IntegerType(), True),
        StructField("VL_TAXA_JURO_AM", DoubleType(), True),
        StructField("VL_TAXA_JURO_AA", DoubleType(), True),
        StructField("dat_ref_carga", StringType(), True),
        StructField("dh_exec", TimestampType(), True),
    ])

df = spark.createDataFrame(df_list, schema=schema)

In [ ]:
df.show(truncate=False)

In [ ]:
df.select("DS_PROD").distinct().show(truncate=False)

In [ ]:
import pyspark.sql.functions as F

df.filter(F.col("NR_POSI") == 1).show(truncate=False)